In [ ]:
from torchvision import models
from torchinfo import summary


model = models.alexnet(weights="AlexNet_Weights.IMAGENET1K_V1")
summary(model, (1, 3, 224, 224), device="cpu")

In [ ]:
with open("../datasets/imagenet_classes.txt", "r") as file:
    classes = file.read().splitlines()
    
print(f"클래스 개수 : {len(classes)}")
print(f"첫 번째 클래스 레이블 : {classes[0]}")

In [ ]:
import torch
from PIL import Image
from torchvision import models, transforms


transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
    ]
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.alexnet(weights="AlexNet_Weights.IMAGENET1K_V1").eval().to(device)

tensors = []
files = ["../datasets/images/airplane.jpg", "../datasets/images/bus.jpg"]
for file in files:
    image = Image.open(file)
    tensors.append(transform(image))

tensors = torch.stack(tensors)
print(f"입력 텐서의 크기 : {tensors.shape}")

In [ ]:
import numpy as np
from torch.nn import functional as F


with torch.no_grad():
    outputs = model(tensors.to(device))
    probs = F.softmax(outputs, dim=-1)
    top_probs, top_idxs = probs.topk(5)

top_probs = top_probs.detach().cpu().numpy()
top_idxs = top_idxs.detach().cpu().numpy()
top_classes = np.array(classes)[top_idxs]

for idx, (cls, prob) in enumerate(zip(top_classes, top_probs)):
    print(f"{files[idx]} 추론 결과")
    for c, p in zip(cls, prob):
        print(f" - {c:<30} : {p * 100:>5.2f}%")